<a href="https://colab.research.google.com/github/JJayy/JJayy.github.io/blob/master/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Function 정의

- seed_everything() : seed 고정 함수
- generate_label() : label 생성 함수
- feature_preprocessing() : 데이터 전처리 함수(label encoding, 결측치 처리)
- feature_selection() : 피처 선택 함수
- feature_engineering2() : 피처 엔지니어링 함수


## Library Import

In [ ]:
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import os, sys, gc, warnings, random

import datetime
import dateutil.relativedelta

# Data manipulation
import pandas as pd 
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb

from tqdm.notebook import trange, tqdm

from IPython.display import display

%matplotlib inline

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

## Fix Seed 함수 정의

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 42
seed_everything(SEED)

## 레이블 생성 함수 정의

- 레이블 생성 함수는 연월을 인자로 받아서 고객 아이디별로 총 구매액이 인자로 받는 구매액 임계값을 넘는지 여부를 바이너리 레이블로 생성하는 함수

In [ ]:
TOTAL_THRES = 340

'''
    입력인자로 받는 year_month에 대해 고객 ID별로 총 구매액이
    구매액 임계값을 넘는지 여부의 binary label을 생성하는 함수
'''
def generate_label(df, year_month, total_thres=TOTAL_THRES, print_log=False):
    df = df.copy()
    
    # year_month에 해당하는 label 데이터 생성
    df['year_month'] = df['order_date'].dt.strftime('%Y-%m')
    df.reset_index(drop=True, inplace=True)

    # year_month 이전 월의 고객 ID 추출
    cust = df[df['year_month']<year_month]['customer_id'].unique()
    # year_month에 해당하는 데이터 선택
    df = df[df['year_month']==year_month]
    
    # label 데이터프레임 생성
    label = pd.DataFrame({'customer_id':cust})
    label['year_month'] = year_month
    
    # year_month에 해당하는 고객 ID의 구매액의 합 계산
    grped = df.groupby(['customer_id','year_month'], as_index=False)[['total']].sum()
    
    # label 데이터프레임과 merge하고 구매액 임계값을 넘었는지 여부로 label 생성
    label = label.merge(grped, on=['customer_id','year_month'], how='left')
    label['total'].fillna(0.0, inplace=True)
    label['label'] = (label['total'] > total_thres).astype(int)

    # 고객 ID로 정렬
    label = label.sort_values('customer_id').reset_index(drop=True)
    if print_log: print(f'{year_month} - final label shape: {label.shape}')
    
    return label

## 데이터 전처리 함수 정의
- 범주형 변수 레이블 인코딩
- 결측치 중위값으로 채움

In [ ]:
def feature_preprocessing(train, test, features, do_imputing=True):
    x_tr = train.copy()
    x_te = test.copy()
    
    # 범주형 피처 이름을 저장할 변수
    cate_cols = []

    # 레이블 인코딩
    for f in features:
        if x_tr[f].dtype.name == 'object': # 데이터 타입이 object(str)이면 레이블 인코딩
            cate_cols.append(f)
            le = LabelEncoder()
            # train + test 데이터를 합쳐서 레이블 인코딩 함수에 fit
            le.fit(list(x_tr[f].values) + list(x_te[f].values))
            
            # train 데이터 레이블 인코딩 변환 수행
            x_tr[f] = le.transform(list(x_tr[f].values))
            
            # test 데이터 레이블 인코딩 변환 수행
            x_te[f] = le.transform(list(x_te[f].values))

    print('categorical feature:', cate_cols)

    if do_imputing:
        # 중위값으로 결측치 채우기
        imputer = SimpleImputer(strategy='median')

        x_tr[features] = imputer.fit_transform(x_tr[features])
        x_te[features] = imputer.transform(x_te[features])

    x_tr = x_tr.drop(columns=['customer_id', 'year_month'])
    x_te = x_te.drop(columns=['customer_id', 'year_month'])
    
    return x_tr, x_te

## 피처 엔지니어링 함수 정의
- Pandas groupby aggregation 함수를 사용한 피처 엔지니어링

In [ ]:
def feature_selection(train, test):
    cor_matrix = train.corr().abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]

    tr = train.drop(columns=to_drop)
    ts = test.drop(columns=to_drop)
    features = tr.columns

    return tr, ts, features


In [ ]:
def feature_engineering2(df, year_month):
    df = df.copy()

    df['month'] = df['order_date'].dt.month
    df['year_month'] = df['order_date'].dt.strftime('%Y-%m')
    
    #
    df['order_ts'] = df['order_date'].astype(np.int64)//1e9
    df['order_ts_diff'] = df.groupby(['customer_id'])['order_ts'].diff()
    df['quantity_diff'] = df.groupby(['customer_id'])['quantity'].diff()
    df['price_diff'] = df.groupby(['customer_id'])['price'].diff()
    df['total_diff'] = df.groupby(['customer_id'])['total'].diff()

    
    # customer_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_cust_id'] = df.groupby(['customer_id'])['total'].cumsum()
    df['cumsum_quantity_by_cust_id'] = df.groupby(['customer_id'])['quantity'].cumsum()
    df['cumsum_price_by_cust_id'] = df.groupby(['customer_id'])['price'].cumsum()

    # product_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_prod_id'] = df.groupby(['product_id'])['total'].cumsum()
    df['cumsum_quantity_by_prod_id'] = df.groupby(['product_id'])['quantity'].cumsum()
    df['cumsum_price_by_prod_id'] = df.groupby(['product_id'])['price'].cumsum()
    
    # order_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_order_id'] = df.groupby(['order_id'])['total'].cumsum()
    df['cumsum_quantity_by_order_id'] = df.groupby(['order_id'])['quantity'].cumsum()
    df['cumsum_price_by_order_id'] = df.groupby(['order_id'])['price'].cumsum()   

    
    # customer_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumcount_total_by_cust_id'] = df.groupby(['customer_id'])['total'].cumcount()
    df['cumcount_quantity_by_cust_id'] = df.groupby(['customer_id'])['quantity'].cumcount()
    df['cumcount_price_by_cust_id'] = df.groupby(['customer_id'])['price'].cumcount()

    # product_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumcount_total_by_prod_id'] = df.groupby(['product_id'])['total'].cumcount()
    df['cumcount_quantity_by_prod_id'] = df.groupby(['product_id'])['quantity'].cumcount()
    df['cumcount_price_by_prod_id'] = df.groupby(['product_id'])['price'].cumcount()
    
    # order_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumcount_total_by_order_id'] = df.groupby(['order_id'])['total'].cumcount()
    df['cumcount_quantity_by_order_id'] = df.groupby(['order_id'])['quantity'].cumcount()
    df['cumcount_price_by_order_id'] = df.groupby(['order_id'])['price'].cumcount()


    
    # year_month 이전 월 계산
    d = datetime.datetime.strptime(year_month, "%Y-%m")
    prev_ym = d - dateutil.relativedelta.relativedelta(months=1)
    prev_ym = prev_ym.strftime('%Y-%m')
    
    # train, test 데이터 선택
    train = df[df['order_date'] < prev_ym]
    test = df[df['order_date'] < year_month]
    
    # train, test 레이블 데이터 생성
    train_label = generate_label(df, prev_ym)[['customer_id','year_month','label']]
    test_label = generate_label(df, year_month)[['customer_id','year_month','label']]
    
    # group by aggregation 함수 선언
    agg_func = ['mean','max','min','sum','count','std','skew','sem', 'var', 'median']
    agg_dict = {
        'quantity': agg_func,
        'price': agg_func,
        'total': agg_func,
        'cumsum_total_by_cust_id': agg_func,
        'cumsum_quantity_by_cust_id': agg_func,
        'cumsum_price_by_cust_id': agg_func,
        'cumsum_total_by_prod_id': agg_func,
        'cumsum_quantity_by_prod_id': agg_func,
        'cumsum_price_by_prod_id': agg_func,
        'cumsum_total_by_order_id': agg_func,
        'cumsum_quantity_by_order_id': agg_func,
        'cumsum_price_by_order_id': agg_func,
        'cumcount_total_by_cust_id': agg_func,
        'cumcount_quantity_by_cust_id': agg_func,
        'cumcount_price_by_cust_id': agg_func,
        'cumcount_total_by_prod_id': agg_func,
        'cumcount_quantity_by_prod_id': agg_func,
        'cumcount_price_by_prod_id': agg_func,
        'cumcount_total_by_order_id': agg_func,
        'cumcount_quantity_by_order_id': agg_func,
        'cumcount_price_by_order_id': agg_func,
        'order_id': ['nunique'],
        'product_id': ['nunique'],
        'month': [lambda x:x.value_counts().index[0]],
        'year_month': [lambda x:x.value_counts().index[0]],
        'order_ts': ['first', 'last'],
        'order_ts_diff': agg_func,
        'quantity_diff': agg_func,
        'price_diff': agg_func,
        'total_diff': agg_func,
    }
    all_train_data = pd.DataFrame()
    
    for i, tr_ym in enumerate(train_label['year_month'].unique()):
        # group by aggretation 함수로 train 데이터 피처 생성
        train_agg = train.loc[train['order_date'] < tr_ym].groupby(['customer_id']).agg(agg_dict)

        new_cols = []
        for col in agg_dict.keys():
            for stat in agg_dict[col]:
                if type(stat) is str:
                    new_cols.append(f'{col}-{stat}')
                else:
                    new_cols.append(f'{col}-mode')

        train_agg.columns = new_cols
        train_agg.reset_index(inplace = True)
        
        train_agg['year_month'] = tr_ym
        
        all_train_data = all_train_data.append(train_agg)
    
    all_train_data = train_label.merge(all_train_data, on=['customer_id', 'year_month'], how='left')
    features = all_train_data.drop(columns=['customer_id', 'label', 'year_month']).columns
    print(len(features))
    # group by aggretation 함수로 test 데이터 피처 생성
    test_agg = test.groupby(['customer_id']).agg(agg_dict)
    test_agg.columns = new_cols
    
    test_data = test_label.merge(test_agg, on=['customer_id'], how='left')

    # feature selection 진행
    all_train_data, test_data, features = feature_selection(all_train_data, test_data)
    features = all_train_data.drop(columns=['customer_id', 'label', 'year_month']).columns

    # train, test 데이터 전처리
    x_tr, x_te = feature_preprocessing(all_train_data, test_data, features)
    
    print('x_tr.shape', x_tr.shape, ', x_te.shape', x_te.shape)
    print('features', len(features))
    
    return x_tr, x_te, all_train_data['label'], features

## Train 데이터 읽기
- 9개의 feature를 가진 780502개의 train data

In [ ]:
data = pd.read_csv("../input/train.csv", parse_dates=["order_date"])
print(data.shape) # data shape 확인
data.head()

(780502, 9)


,order_id,product_id,description,quantity,order_date,price,customer_id,country,total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,11.4675,13085,United Kingdom,137.61
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,3.4650,13085,United Kingdom,166.32
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,2.0625,13085,United Kingdom,49.50


## Pandas info() 함수로 데이터 타입 및 null 체크

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780502 entries, 0 to 780501
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   order_id     780502 non-null  object        
 1   product_id   780502 non-null  object        
 2   description  780502 non-null  object        
 3   quantity     780502 non-null  int64         
 4   order_date   780502 non-null  datetime64[ns]
 5   price        780502 non-null  float64       
 6   customer_id  780502 non-null  int64         
 7   country      780502 non-null  object        
 8   total        780502 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 53.6+ MB


## Pandas isna(), sum() 함수로 null 데이터 개수 체크

In [ ]:
data.isna().sum()

order_id       0
product_id     0
description    0
quantity       0
order_date     0
price          0
customer_id    0
country        0
total          0
dtype: int64

## Pandas describe() 함수에 include='all' 인자 설정으로 수치형, 범주형 데이터 기본 통계량 확인

In [ ]:
data.describe(include='all')

,order_id,product_id,description,quantity,order_date,price,customer_id,country,total
count,780502,780502,780502,780502.000000,780502,780502.000000,780502.000000,780502,780502.000000
unique,43955,4645,5297,NaN,40615,NaN,NaN,41,NaN
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-11-14 15:27:00,NaN,NaN,United Kingdom,NaN
freq,542,5096,5096,NaN,543,NaN,NaN,700317,NaN
first,NaN,NaN,NaN,NaN,2009-12-01 07:45:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-11-30 17:42:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,12.623307,NaN,6.140931,15313.393466,NaN,33.715373
std,NaN,NaN,NaN,143.906302,NaN,119.042266,1695.765797,NaN,274.207403
min,NaN,NaN,NaN,-74215.000000,NaN,0.000000,12346.000000,NaN,-127352.940000
25%,NaN,NaN,NaN,2.000000,NaN,2.062500,13963.000000,NaN,7.260000


# 모델 학습

In [ ]:
from pycaret.classification import *

In [ ]:
train, test, y, features = feature_engineering2(data, '2011-12')
features

256
categorical feature: ['year_month-mode']
x_tr.shape (5722, 69) , x_te.shape (5914, 69)
features 68


Index(['quantity-mean', 'quantity-max', 'quantity-sum', 'quantity-count',
       'quantity-skew', 'price-mean', 'price-max', 'price-sum', 'price-skew',
       'total-skew', 'cumsum_total_by_cust_id-sum',
       'cumsum_total_by_cust_id-skew', 'cumsum_total_by_cust_id-sem',
       'cumsum_quantity_by_cust_id-min', 'cumsum_quantity_by_cust_id-skew',
       'cumsum_price_by_cust_id-skew', 'cumsum_total_by_prod_id-mean',
       'cumsum_total_by_prod_id-max', 'cumsum_total_by_prod_id-min',
       'cumsum_total_by_prod_id-std', 'cumsum_total_by_prod_id-skew',
       'cumsum_total_by_prod_id-sem', 'cumsum_quantity_by_prod_id-mean',
       'cumsum_quantity_by_prod_id-max', 'cumsum_quantity_by_prod_id-min',
       'cumsum_quantity_by_prod_id-skew', 'cumsum_quantity_by_prod_id-sem',
       'cumsum_price_by_prod_id-mean', 'cumsum_price_by_prod_id-std',
       'cumsum_price_by_prod_id-skew', 'cumsum_total_by_order_id-max',
       'cumsum_total_by_order_id-skew', 'cumsum_quantity_by_order_id-mean',

In [ ]:
train

,label,quantity-mean,quantity-max,quantity-sum,quantity-count,quantity-skew,price-mean,price-max,price-sum,price-skew,total-skew,cumsum_total_by_cust_id-sum,cumsum_total_by_cust_id-skew,cumsum_total_by_cust_id-sem,cumsum_quantity_by_cust_id-min,cumsum_quantity_by_cust_id-skew,cumsum_price_by_cust_id-skew,cumsum_total_by_prod_id-mean,cumsum_total_by_prod_id-max,cumsum_total_by_prod_id-min,cumsum_total_by_prod_id-std,cumsum_total_by_prod_id-skew,cumsum_total_by_prod_id-sem,cumsum_quantity_by_prod_id-mean,cumsum_quantity_by_prod_id-max,cumsum_quantity_by_prod_id-min,cumsum_quantity_by_prod_id-skew,cumsum_quantity_by_prod_id-sem,cumsum_price_by_prod_id-mean,cumsum_price_by_prod_id-std,cumsum_price_by_prod_id-skew,cumsum_total_by_order_id-max,cumsum_total_by_order_id-skew,cumsum_quantity_by_order_id-mean,cumsum_quantity_by_order_id-max,cumsum_quantity_by_order_id-min,cumsum_quantity_by_order_id-skew,cumsum_price_by_order_id-sum,cumsum_price_by_order_id-skew,cumcount_total_by_cust_id-min,cumcount_total_by_cust_id-skew,cumcount_quantity_by_cust_id-min,cumcount_quantity_by_cust_id-skew,cumcount_price_by_cust_id-min,cumcount_price_by_cust_id-skew,cumcount_total_by_prod_id-min,cumcount_total_by_prod_id-skew,cumcount_total_by_order_id-mean,cumcount_total_by_order_id-min,cumcount_total_by_order_id-skew,cumcount_total_by_order_id-sem,cumcount_quantity_by_order_id-min,cumcount_price_by_order_id-min,month-mode,year_month-mode,order_ts-first,order_ts-last,order_ts_diff-mean,order_ts_diff-max,order_ts_diff-min,order_ts_diff-sum,order_ts_diff-skew,quantity_diff-mean,quantity_diff-skew,price_diff-mean,price_diff-sum,price_diff-skew,total_diff-mean,total_diff-skew
0,0,1.127660,74215.0,53.0,47.0,-0.000228,19.922872,234.8115,936.3750,4.547689,0.000214,1.388053e+05,6.855058,2702.559040,10.0,6.855636,0.440160,-18933.867255,127352.9400,-145015.2000,64968.445670,-1.131166,9476.621775,2354.212766,74215.0,-1644.0,6.742285,1570.615648,63650.017691,128903.599171,1.619409,127352.9400,-0.002867,3.914894,74215.0,-74215.0,-0.000793,3794.6700,1.486909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.736555,4.617021,0.0,1.072454,0.787596,0.0,0.0,6.0,6.0,1.260780e+09,1.295346e+09,751438.695652,10197900.0,0.0,34566180.0,3.439852,-1613.586957,-4.214331,-1.241087e-01,-5.709000e+00,-0.840908,-2.770156e+03,-4.214047
1,0,13.151659,240.0,2775.0,211.0,10.216574,4.230803,21.0375,892.6995,2.511623,6.436852,7.367592e+05,0.228741,158.497385,6.0,0.085304,0.107158,16340.371493,406502.2830,156.3375,47293.442526,6.402293,3255.814853,5018.199052,50996.0,56.0,2.895244,574.065075,2199.501924,7974.127747,6.790150,2135.6280,1.234591,224.853081,676.0,4.0,0.694310,15086.5935,0.663454,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.453315,16.075829,0.0,0.607186,0.767643,0.0,0.0,10.0,22.0,1.288535e+09,1.320064e+09,150138.571429,7789020.0,0.0,31529100.0,6.372104,0.000000,0.304318,-7.857143e-03,-1.650000e+00,0.840331,-4.714286e-02,0.041801
2,0,53.215686,144.0,2714.0,51.0,0.822147,6.247353,66.0000,318.6150,3.224035,2.528198,5.594629e+04,0.812596,140.030692,24.0,0.493378,1.597336,17500.335471,146857.4085,471.2400,27894.026806,3.120231,3905.945020,11485.372549,35932.0,336.0,0.673806,1548.160716,5104.847000,16507.639799,4.106796,1473.1200,1.639190,388.392157,1254.0,24.0,1.339273,719.8950,2.593528,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.186634,6.941176,0.0,0.527392,0.791827,0.0,0.0,9.0,9.0,1.285600e+09,1.316956e+09,627136.800000,14955960.0,0.0,31356840.0,4.710186,-0.460000,0.066540,1.301850e+00,6.509250e+01,0.251831,8.844000e-01,-0.069972
3,1,9.233645,48.0,988.0,107.0,1.558634,13.870640,412.5000,1484.1585,7.102276,4.889357,2.155245e+05,0.089354,135.620199,-5.0,-0.077728,0.447134,8876.528131,86587.3965,74.2500,12932.316422,3.313619,1250.214217,2352.869159,33089.0,12.0,4.872195,410.319118,1555.892636,3418.294904,6.236526,2314.3230,0.321784,224.383178,519.0,-5.0,0.244650,25612.9170,1.326011,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.666999,23.644860,0.0,0.126067,1.501146,0.0,0.0,10.0,10.0,1.259931e+09,1.288254e+09,267200.377358,14077560.0,0.0,28323240.0,7

In [ ]:
clf = setup(data=train, target='label')

,Description,Value
0,session_id,1974
1,Target,label
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(5722, 69)"
5,Missing Values,False
6,Numeric Features,68
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best=compare_models(sort='AUC', n_select= 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8302,0.8245,0.3769,0.6555,0.4756,0.3840,0.4061,6.3690
gbc,Gradient Boosting Classifier,0.8257,0.8172,0.3587,0.6388,0.4566,0.3635,0.3860,1.2160
lda,Linear Discriminant Analysis,0.8195,0.8125,0.3575,0.6160,0.4500,0.3510,0.3710,0.0310
rf,Random Forest Classifier,0.8272,0.8124,0.3237,0.6705,0.4334,0.3469,0.3801,0.4240
et,Extra Trees Classifier,0.8252,0.8122,0.3165,0.6590,0.4251,0.3380,0.3702,0.2850
lightgbm,Light Gradient Boosting Machine,0.8227,0.8113,0.3866,0.6115,0.4715,0.3723,0.3873,0.1920
ada,Ada Boost Classifier,0.8145,0.8013,0.3925,0.5754,0.4656,0.3585,0.3686,0.2740
xgboost,Extreme Gradient Boosting,0.8180,0.7979,0.3877,0.5929,0.4670,0.3634,0.3762,12.2370
nb,Naive Bayes,0.8067,0.7796,0.2560,0.5885,0.3517,0.2587,0.2919,0.0190
qda,Quadratic Discriminant Analysis,0.8085,0.7755,0.2946,0.5735,0.3851,0.2866,0.3105,0.0180


In [ ]:
blended = blend_models(estimator_list=[best[0], best[1]], fold = 5, method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8102,0.8114,0.3212,0.5699,0.4109,0.3081,0.3261
1,0.8290,0.8385,0.2909,0.7059,0.4120,0.3317,0.3765
2,0.8315,0.8193,0.3614,0.6742,0.4706,0.3810,0.4073
3,0.8365,0.8303,0.4217,0.6667,0.5166,0.4241,0.4402
4,0.8277,0.8212,0.4277,0.6228,0.5071,0.4071,0.4176
Mean,0.8270,0.8242,0.3646,0.6479,0.4634,0.3704,0.3936
SD,0.0089,0.0094,0.0540,0.0472,0.0452,0.0441,0.0395


In [ ]:
pred_holdout=predict_model(blended2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8480,0.8270,0.3726,0.6464,0.4727,0.3913,0.4116


In [ ]:
final_model=finalize_model(blended)

In [ ]:
predictions=predict_model(final_model, data=test)

In [ ]:
predictions

,label,quantity-mean,quantity-max,quantity-sum,quantity-count,quantity-skew,price-mean,price-max,price-sum,price-skew,...,order_ts_diff-skew,quantity_diff-mean,quantity_diff-skew,price_diff-mean,price_diff-sum,price_diff-skew,total_diff-mean,total_diff-skew,Label,Score
0,0,1.127660,74215.0,53.0,47.0,-0.000228,19.922872,234.8115,936.3750,4.547689,...,3.439852,-1613.586957,-4.214331,-1.241087e-01,-5.709000e+00,-0.840908,-2.770156e+03,-4.214047,0,0.9625
1,0,13.151659,240.0,2775.0,211.0,10.216574,4.230803,21.0375,892.6995,2.511623,...,6.372104,0.000000,0.304318,-7.857143e-03,-1.650000e+00,0.840331,-4.714286e-02,0.041801,0,0.7584
2,0,53.215686,144.0,2714.0,51.0,0.822147,6.247353,66.0000,318.6150,3.224035,...,4.710186,-0.460000,0.066540,1.301850e+00,6.509250e+01,0.251831,8.844000e-01,-0.069972,0,0.9185
3,0,8.994444,48.0,1619.0,180.0,1.718638,13.792075,495.0000,2482.5735,7.661229,...,9.890780,0.011173,-0.099464,2.730796e+00,4.888125e+02,1.905593,2.799930e+00,1.341806,0,0.8969
4,0,11.588235,24.0,197.0,17.0,0.472554,6.337941,66.0000,107.7450,4.098028,...,0.000000,0.000000,-0.831202,4.440892e-16,7.105427e-15,0.079587,2.220446e-16,0.404341,0,0.9229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,0,1.730856,13.0,1537.0,888.0,4.464300,2.947368,26.3175,2617.2630,3.900581,...,14.044305,0.010147,0.138229,1.543968e-03,1.369500e+00,0.277995,3.636697e-02,0.573504,1,0.7755
5910,0,17.000000,50.0,493.0,29.0,1.006840,6.605121,82.5000,191.5485,4.144780,...,5.291503,-0.071429,-0.256523,-1.473214e+00,-4.125000e+01,-3.123508,-4.419643e+00,-2.121658,0,0.9499
5911,0,12.083333,80.0,145.0,12.0,3.258881,13.777500,82.5000,165.3300,2.931939,...,0.000000,-0.090909,0.008851,5.257500e+00,5.783250e+01,2.748160,3.015000e+00,0.195896,0,0.9845
5912,0,8.457143,36.0,592.0,70.0,1.525244,7.225821,82.5000,505.8075,4.944969,...,7.469056,-0.144928,0.525894,1.492174e-01,1.029600e+01,0.599378,-4.782609e-04,-0.130166,0,0.9383


In [ ]:
predictions['Score']=np.where(predictions['Label']==0, 1-predictions['Score'], predictions['Score'])
predictions

,label,quantity-mean,quantity-max,quantity-sum,quantity-count,quantity-skew,price-mean,price-max,price-sum,price-skew,...,order_ts_diff-skew,quantity_diff-mean,quantity_diff-skew,price_diff-mean,price_diff-sum,price_diff-skew,total_diff-mean,total_diff-skew,Label,Score
0,0,1.127660,74215.0,53.0,47.0,-0.000228,19.922872,234.8115,936.3750,4.547689,...,3.439852,-1613.586957,-4.214331,-1.241087e-01,-5.709000e+00,-0.840908,-2.770156e+03,-4.214047,0,0.0375
1,0,13.151659,240.0,2775.0,211.0,10.216574,4.230803,21.0375,892.6995,2.511623,...,6.372104,0.000000,0.304318,-7.857143e-03,-1.650000e+00,0.840331,-4.714286e-02,0.041801,0,0.2416
2,0,53.215686,144.0,2714.0,51.0,0.822147,6.247353,66.0000,318.6150,3.224035,...,4.710186,-0.460000,0.066540,1.301850e+00,6.509250e+01,0.251831,8.844000e-01,-0.069972,0,0.0815
3,0,8.994444,48.0,1619.0,180.0,1.718638,13.792075,495.0000,2482.5735,7.661229,...,9.890780,0.011173,-0.099464,2.730796e+00,4.888125e+02,1.905593,2.799930e+00,1.341806,0,0.1031
4,0,11.588235,24.0,197.0,17.0,0.472554,6.337941,66.0000,107.7450,4.098028,...,0.000000,0.000000,-0.831202,4.440892e-16,7.105427e-15,0.079587,2.220446e-16,0.404341,0,0.0771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,0,1.730856,13.0,1537.0,888.0,4.464300,2.947368,26.3175,2617.2630,3.900581,...,14.044305,0.010147,0.138229,1.543968e-03,1.369500e+00,0.277995,3.636697e-02,0.573504,1,0.7755
5910,0,17.000000,50.0,493.0,29.0,1.006840,6.605121,82.5000,191.5485,4.144780,...,5.291503,-0.071429,-0.256523,-1.473214e+00,-4.125000e+01,-3.123508,-4.419643e+00,-2.121658,0,0.0501
5911,0,12.083333,80.0,145.0,12.0,3.258881,13.777500,82.5000,165.3300,2.931939,...,0.000000,-0.090909,0.008851,5.257500e+00,5.783250e+01,2.748160,3.015000e+00,0.195896,0,0.0155
5912,0,8.457143,36.0,592.0,70.0,1.525244,7.225821,82.5000,505.8075,4.944969,...,7.469056,-0.144928,0.525894,1.492174e-01,1.029600e+01,0.599378,-4.782609e-04,-0.130166,0,0.0617


# 제출 파일 생성

In [ ]:
output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)
submission = pd.read_csv('../input/sample_submission.csv')
submission['probability'] = predictions['Score']
submission.to_csv(os.path.join(output_dir, 'output.csv'), index=False)